# Языковое моделирование

## Импорты

In [ ]:
import numpy as np
import tensorflow as tf

## Настройки

In [ ]:
PATH = './evgenyi_onegin.txt'

SEQ_LENGTH = 100

EPOCHS = 15
BATCH_SIZE = 32
BUFFER_SIZE = 30000

RNN_UNITS = 1024
EMBEDDING_DIM = 256

## Данные

In [ ]:
with open(PATH, 'r', encoding='utf-8') as file:
    text = file.read()

text = '\n'.join(line.strip() for line in text.split('\n'))

print(len(text))
print(text[:500])

143928
Александр Сергеевич Пушкин

Евгений Онегин
Роман в стихах

Не мысля гордый свет забавить,
Вниманье дружбы возлюбя,
Хотел бы я тебе представить
Залог достойнее тебя,
Достойнее души прекрасной,
Святой исполненной мечты,
Поэзии живой и ясной,
Высоких дум и простоты;
Но так и быть - рукой пристрастной
Прими собранье пестрых глав,
Полусмешных, полупечальных,
Простонародных, идеальных,
Небрежный плод моих забав,
Бессонниц, легких вдохновений,
Незрелых и увядших лет,
Ума холодных наблюдений
И сердца го


In [ ]:
vocab = sorted(set(text))
char2idx = {char: i for i, char in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[char] for char in text])
print(len(text_as_int), len(text))

143928 143928


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(SEQ_LENGTH+1, drop_remainder=True)
dataset = sequences.map(split_input_target)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Модель

In [ ]:
# без задания батч-сайза
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
vocab_size = len(vocab)
model = build_model(vocab_size, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.compile(optimizer='adam', loss=loss)

In [ ]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/15
44/44 [==============================] - 19s 176ms/step - loss: 3.8499
Epoch 2/15
44/44 [==============================] - 7s 146ms/step - loss: 3.0091
Epoch 3/15
44/44 [==============================] - 6s 122ms/step - loss: 2.5826
Epoch 4/15
44/44 [==============================] - 6s 124ms/step - loss: 2.4253
Epoch 5/15
44/44 [==============================] - 6s 131ms/step - loss: 2.3076
Epoch 6/15
44/44 [==============================] - 6s 122ms/step - loss: 2.2023
Epoch 7/15
44/44 [==============================] - 6s 122ms/step - loss: 2.0874
Epoch 8/15
44/44 [==============================] - 6s 122ms/step - loss: 1.9657
Epoch 9/15
44/44 [==============================] - 6s 122ms/step - loss: 1.8274
Epoch 10/15
44/44 [==============================] - 6s 123ms/step - loss: 1.6820
Epoch 11/15
44/44 [==============================] - 6s 121ms/step - loss: 1.5229
Epoch 12/15
44/44 [==============================] - 6s 122ms/step - loss: 1.3609
Epoch 13/15
44/44 [=====

In [ ]:
def generate_text(model, start_string):
  
    num_generate = 512
    input_eval = [char2idx[char] for char in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
  
    temperature = 0.5

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
       
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
gen_text = generate_text(model, start_string="Случайный текст ")
print(len(gen_text), gen_text, sep='\n')

528
Случайный текст в в хокрадододо слала но до доводонараралине ве мнето ви ве пой пой пой хотодовай по о о дамисой сто пе мне возв вымлитовой ходамной ный от,
Пе желе ми во боде пле сть поче далоть вой м вой ветотой в,
И стовой наво воклинозакотобоной хотодотой мны наль сокодей ноде от в ой блой ны
Пой вымара пой и дастой подракакасторе пой ви о отася водна ной о залодалой дарить оде сям,
Водо бый однаритой онареть ои оде хой вой сцы но накой о ся вовымнетоть вой вокотой в де вить ви нудралисодрестова пралы ся ло драпой м,
Ч
